In [3]:
ENV_LIST = ['Pendulum-v0', 'MountainCar-v0']
ENV_IDX = 0

In [37]:
import numpy as np
import random

import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
env = gym.make(ENV_LIST[ENV_IDX])

/home/hogun/anaconda2/envs/gym/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
actions = [4/10*i-2 for i in range(11)]

In [7]:
actions

[-2.0,
 -1.6,
 -1.2,
 -0.7999999999999998,
 -0.3999999999999999,
 0.0,
 0.40000000000000036,
 0.8000000000000003,
 1.2000000000000002,
 1.6,
 2.0]

In [8]:
print(env.observation_space)
print(env.action_space)

Box(3,)
Box(1,)


In [10]:
def get_action(q_net, inputs, epsilon):
    # randomly choose
    if random.random() <= epsilon:
        action = random.sample(actions, 1)
    else:
        action = q_net(inputs)

In [83]:
class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        self.fc1 = nn.Linear(3, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 11)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [105]:
epsilon = 1.0
min_epsilon = 0.1

replay_buffer = []

In [106]:
max_buff_size = 100
batch_size = 32
gamma = 0.999

In [137]:
q_net = QNet()
target_net = QNet()
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(q_net.parameters(), lr=0.00025, momentum=0.95)

done = False
t_step = 0
ridx = 0
observation = env.reset()

while t_step<100 or not done:
    #env.render()
    pre_obs = observation
    #action = env.action_space.sample()
    q_value = q_net(torch.tensor(observation, dtype=torch.float32))
    #print('Q value: ', q_value)
    action_idx = np.argmax(q_value.tolist())
    print('selected action:', action_idx)
    
    observation, reward, done, info = env.step([actions[action_idx]])
    
    if len(replay_buffer)<max_buff_size:
        replay_buffer.append([pre_obs, action_idx, reward, observation, done])
        if len(replay_buffer)<batch_size:
            continue
    else:
        replay_buffer[ridx%max_buff_size] = [pre_obs, action_idx, reward, observation, done]
        ridx += 1
    
    minibatch = np.array(random.sample(replay_buffer, batch_size))
    y_target = np.expand_dims((minibatch[:, 2]), axis=1) + gamma * np.expand_dims((1-minibatch[:, 4]), axis=1) * target_net(torch.tensor(np.concatenate(minibatch[:,3], axis=0).reshape(-1, 3), dtype=torch.float32)).detach().numpy()
    q_values = q_net(torch.tensor(np.concatenate(minibatch[:,0], axis=0).reshape(-1, 3), dtype=torch.float32))
    loss = criterion(torch.tensor(y_target.astype(np.float32)), q_values)
    
    loss.backward()
    optimizer.step()
    
    print(loss)
    #running_loss += loss.item()

selected action: 4
tensor(65.9640, grad_fn=<MeanBackward0>)
selected action: 4
tensor(87.2574, grad_fn=<MeanBackward0>)
selected action: 4
tensor(82.0952, grad_fn=<MeanBackward0>)
selected action: 4
tensor(94.2554, grad_fn=<MeanBackward0>)
selected action: 8
tensor(97.9001, grad_fn=<MeanBackward0>)
selected action: 8
tensor(67.2686, grad_fn=<MeanBackward0>)
selected action: 8
tensor(71.6699, grad_fn=<MeanBackward0>)
selected action: 8
tensor(71.9877, grad_fn=<MeanBackward0>)
selected action: 8
tensor(63.8054, grad_fn=<MeanBackward0>)
selected action: 8
tensor(37.2022, grad_fn=<MeanBackward0>)
selected action: 8
tensor(47.0379, grad_fn=<MeanBackward0>)
selected action: 9
tensor(29.0043, grad_fn=<MeanBackward0>)
selected action: 9
tensor(13.3576, grad_fn=<MeanBackward0>)
selected action: 9
tensor(15.7124, grad_fn=<MeanBackward0>)
selected action: 9
tensor(64.0023, grad_fn=<MeanBackward0>)
selected action: 9
tensor(191.8394, grad_fn=<MeanBackward0>)
selected action: 9
tensor(387.4346, gra

KeyboardInterrupt: 